In [1]:
from __future__ import print_function
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
from keras.callbacks import ModelCheckpoint
import numpy as np
import random
import sys

Using TensorFlow backend.


In [2]:
path = get_file('data.txt', origin='https://gist.githubusercontent.com/DakshMiglani/a51e2c70d20f4d048521440c7b1a421f/raw/1b3240c63714a23626f2548d2f3ca61d50fc5acb/data.txt')
text = open(path).read().lower()
print('Text Length:', len(text))

Text Length: 40299


In [3]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

total chars: 36


In [4]:
maxlen = 70
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('Total Sequences:', len(sentences))

Total Sequences: 13410


In [5]:
print('Vectorizing Text...')
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Vectorizing Text...


In [6]:
print('Build model...')
model = Sequential()
model.add(LSTM(256, input_shape=(maxlen, len(chars))))
model.add(Dropout(0.2))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Build model...


In [7]:
filepath="weights/Chester Loss {loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [8]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [9]:
def genText(start_index, maxlen):
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print()
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x[0, t, char_indices[char]] = 1.

            preds = model.predict(x, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

In [49]:
def lyric(start_index, maxlen, c):
    for diversity in c:
        print()
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
#         print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(650):
            x = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x[0, t, char_indices[char]] = 1.

            preds = model.predict(x, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

In [10]:
def train_and_gen(epochs, end):
    for iteration in range(1, epochs):
        print()
        print('-' * 50)
        print('Iteration', iteration)
        model.fit(X, y,
                  batch_size=128,
                  epochs=1,
                 callbacks=callbacks_list)

        start_index = random.randint(0, len(text) - maxlen - 1)
        if iteration % end == 0:
            genText(start_index, maxlen)

In [11]:
def load_model(fin):
    model.load_weights(fin)
    model.compile(loss='categorical_crossentropy', optimizer='adam')

In [12]:
load_model('weights/Chester Loss 0.0043.hdf5')

In [51]:
lyric(random.randint(0, len(text) - maxlen - 1), maxlen, [0.25])


----- diversity: 0.25
ing on
so much more than i can carry
i keep dragging around what's bringing me down
if i just let go, i'd be set free
holding on
why is everything so heavy?

who crouss in the undertow
and i want to feel, whet i take in the undertow, just caught in the undertow
and i wat the fuck is wrong with me!
but you keep spinning 'round me just the same
and fan the flames
as your blazes burn

and i  the pain anot the pain the pain anot the pain the pain and the ede

i know i'm not the center of the universe
but you keep spinning 'round me just the same
and fan the flames
as your blazes burn

and i  the fact inside is how it feels no haring to heal

i fell the go
let it go

a light that never comes

when the face inside is
